In [2]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [5]:
data = pd.read_csv('dataset.csv')
data.head()

,image,label
0,Dataset/2_241_gel.wav.png,gel
1,Dataset/2_580_voi.wav.png,voi
2,Dataset/0_214_pia_sax.wav.png,pia
3,Dataset/0_198_voi.wav.png,voi
4,Dataset/0_144_gel_voi.wav.png,gel


In [15]:
train_im = data['image'][:1200]
test_im = data['image'][1200:]
train_y = data['label'][:1200]
test_y = data['label'][1200:]

In [16]:
train = pd.DataFrame()
test = pd.DataFrame()
train['image'] = train_im
test['image'] = test_im
train['label'] = train_y
test['label'] = test_y

In [40]:
train.to_csv('training.csv', header=True, index = False)
test.to_csv('testing.csv', header = True, index = False)

In [43]:
train_images = []
for i in tqdm(range(1200)):
    im = image.load_img(train['image'][i], target_size = (224, 224))
    im = image.img_to_array(im)
    im = im/255
    train_images.append(im)

100%|██████████| 1200/1200 [00:02<00:00, 561.49it/s]


In [44]:
X = np.array(train_images)

In [45]:
y = train['label']

In [46]:
y.shape

(1200,)

In [47]:
X_train, X_test,y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y) 

In [48]:
X.shape

(1200, 224, 224, 3)

In [49]:
y.shape

(1200,)

In [50]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [51]:
y_train.shape

(960, 11)

In [52]:
y_train

,cel,cla,flu,gac,gel,org,pia,sax,tru,vio,voi
456,0,0,0,0,0,0,0,0,0,0,1
987,0,0,0,0,1,0,0,0,0,0,0
461,0,0,0,0,1,0,0,0,0,0,0
1153,0,0,0,1,0,0,0,0,0,0,0
1022,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1,0,0,0,0,0,0,0,0,0,0,1
806,0,0,0,0,0,0,0,0,0,0,1
877,0,0,0,0,1,0,0,0,0,0,0
496,0,0,0,1,0,0,0,0,0,0,0


In [53]:
basemodel = VGG16(weights = 'imagenet', include_top = False)

In [54]:
X_train = basemodel.predict(X_train)

In [55]:
X_train.shape

(960, 7, 7, 512)

In [56]:
X_test = basemodel.predict(X_test)

In [57]:
X_test.shape

(240, 7, 7, 512)

In [58]:
X_train = X_train.reshape(960, 7*7*512)
X_test = X_test.reshape(240, 7*7*512)

In [59]:
maxi = X_train.max()
X_train = X_train/maxi
X_test = X_test/maxi

In [60]:
X_train.shape

(960, 25088)

In [61]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))

In [62]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [63]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [64]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Train on 960 samples, validate on 240 samples
Epoch 1/100
960/960 [==============================] - 1s 1ms/step - loss: 2.2031 - accuracy: 0.2427 - val_loss: 1.9809 - val_accuracy: 0.3292
Epoch 2/100
960/960 [==============================] - 1s 1ms/step - loss: 2.0180 - accuracy: 0.2958 - val_loss: 2.0382 - val_accuracy: 0.2500
Epoch 3/100
960/960 [==============================] - 1s 1ms/step - loss: 2.0087 - accuracy: 0.2750 - val_loss: 1.9922 - val_accuracy: 0.3292
Epoch 4/100
960/960 [==============================] - 1s 1ms/step - loss: 1.9429 - accuracy: 0.2917 - val_loss: 1.9753 - val_accuracy: 0.4250
Epoch 5/100
960/960 [==============================] - 1s 1ms/step - loss: 1.9146 - accuracy: 0.2823 - val_loss: 1.9279 - val_accuracy: 0.4083
Epoch 6/100
960/960 [==============================] - 1s 1ms/step - loss: 1.8890 - accuracy: 0.3094 - val_loss: 1.9000 - val_accuracy: 0.4292
Epoch 7/100
960/960 [==============================] - 1s 1ms/step - loss: 1.8993 - accuracy: 0.

960/960 [==============================] - 1s 1ms/step - loss: 1.0137 - accuracy: 0.6396 - val_loss: 1.2971 - val_accuracy: 0.5917
Epoch 58/100
960/960 [==============================] - 1s 1ms/step - loss: 0.9956 - accuracy: 0.6562 - val_loss: 1.3419 - val_accuracy: 0.6250
Epoch 59/100
960/960 [==============================] - 1s 1ms/step - loss: 0.9812 - accuracy: 0.6792 - val_loss: 1.3371 - val_accuracy: 0.6208
Epoch 60/100
960/960 [==============================] - 1s 1ms/step - loss: 1.0430 - accuracy: 0.6302 - val_loss: 1.4752 - val_accuracy: 0.5833
Epoch 61/100
960/960 [==============================] - 1s 1ms/step - loss: 0.9973 - accuracy: 0.6625 - val_loss: 1.4036 - val_accuracy: 0.6042
Epoch 62/100
960/960 [==============================] - 1s 1ms/step - loss: 1.0573 - accuracy: 0.6375 - val_loss: 1.2617 - val_accuracy: 0.6417
Epoch 63/100
960/960 [==============================] - 1s 1ms/step - loss: 1.0658 - accuracy: 0.6167 - val_loss: 1.3994 - val_accuracy: 0.6083
Epoch